In [ ]:
!nvidia-smi

In [2]:
!pip install torch transformers datasets accelerate trl peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [77]:
import torch
import transformers
import pandas as pd
import json
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [4]:
# Load UltraFeedback dataset
ultra_feedback = load_dataset("openbmb/UltraFeedback")
print("UltraFeedback dataset structure:")
print(ultra_feedback)

# Load UltraSafety dataset
ultra_safety = load_dataset("openbmb/UltraSafety")
print("UltraSafety dataset structure:")
print(ultra_safety)

README.md:   0%|          | 0.00/15.4k [00:00<?, ?B/s]

evol_instruct.jsonl:   0%|          | 0.00/168M [00:00<?, ?B/s]

false_qa.jsonl:   0%|          | 0.00/25.9M [00:00<?, ?B/s]

flan.jsonl:   0%|          | 0.00/240M [00:00<?, ?B/s]

sharegpt.jsonl:   0%|          | 0.00/313M [00:00<?, ?B/s]

...

UltraFeedback dataset structure:
DatasetDict({
    train: Dataset({
        features: ['source', 'instruction', 'models', 'completions', 'correct_answers', 'incorrect_answers'],
        num_rows: 63967
    })
})


README.md:   0%|          | 0.00/18.1k [00:00<?, ?B/s]

UltraSafety.jsonl:   0%|          | 0.00/87.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3000 [00:00<?, ? examples/s]

UltraSafety dataset structure:
DatasetDict({
    train: Dataset({
        features: ['source', 'prompt', 'instruction', 'models', 'completions'],
        num_rows: 3000
    })
})


In [53]:
# Display column names for UltraFeedback
print("UltraFeedback Columns:", ultra_feedback["train"].column_names)

# Display column names for UltraSafety
print("UltraSafety Columns:", ultra_safety["train"].column_names)

# Show the first few rows of each dataset
print("\nUltraFeedback Sample:")
print(ultra_feedback["train"][0])

print("\nUltraSafety Sample:")
print(ultra_safety["train"][0])


UltraFeedback Columns: ['source', 'instruction', 'models', 'completions', 'correct_answers', 'incorrect_answers']
UltraSafety Columns: ['source', 'prompt', 'instruction', 'models', 'completions']

UltraFeedback Sample:
{'source': 'evol_instruct', 'instruction': 'Can you write a C++ program that prompts the user to enter the name of a country and checks if it borders the Mediterranean Sea? Here\'s some starter code to help you out:\n#include <iostream>\n#include <string>\nusing namespace std;\nint main() {\n    string country;\n    // prompt user for input\n    cout << "Enter the name of a country: ";\n    cin >> country;\n    // check if country borders the Mediterranean Sea\n    // [C++ code]\n    return 0;\n}', 'models': ['alpaca-7b', 'pythia-12b', 'starchat', 'vicuna-33b'], 'completions': [{'annotations': {'helpfulness': {'Rating': '2', 'Rationale': 'The response is clear and not lengthy, but it lacks useful and comprehensive information.', 'Rationale For Rating': 'The code is parti

In [30]:

prism_survey = load_dataset("HannahRoseKirk/prism-alignment", "survey")
print(prism_survey)


survey.jsonl:   0%|          | 0.00/4.86M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['user_id', 'survey_only', 'num_completed_conversations', 'consent', 'consent_age', 'lm_familiarity', 'lm_indirect_use', 'lm_direct_use', 'lm_frequency_use', 'self_description', 'system_string', 'age', 'gender', 'employment_status', 'education', 'marital_status', 'english_proficiency', 'study_id', 'study_locale', 'religion', 'ethnicity', 'location', 'lm_usecases', 'stated_prefs', 'order_lm_usecases', 'order_stated_prefs', 'generated_datetime', 'timing_duration_s', 'timing_duration_mins', 'included_in_US_REP', 'included_in_UK_REP', 'included_in_balanced_subset'],
        num_rows: 1500
    })
})


In [9]:
prism_conversations = load_dataset("HannahRoseKirk/prism-alignment", "conversations")
print(prism_conversations)

conversations.jsonl:   0%|          | 0.00/60.3M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['conversation_id', 'user_id', 'conversation_type', 'opening_prompt', 'conversation_turns', 'conversation_history', 'performance_attributes', 'choice_attributes', 'open_feedback', 'generated_datetime', 'timing_duration_s', 'timing_duration_mins', 'included_in_balanced_subset'],
        num_rows: 8011
    })
})


In [5]:
prism_utterances = load_dataset("HannahRoseKirk/prism-alignment", "utterances")
print(prism_utterances)


README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

utterances.jsonl:   0%|          | 0.00/68.2M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['utterance_id', 'interaction_id', 'conversation_id', 'user_id', 'turn', 'within_turn_id', 'conversation_type', 'user_prompt', 'model_response', 'model_name', 'model_provider', 'score', 'if_chosen', 'included_in_balanced_subset'],
        num_rows: 68371
    })
})


In [6]:
print(prism_utterances["train"][0])  # Print the first example from the dataset

{'utterance_id': 'ut0', 'interaction_id': 'int0', 'conversation_id': 'c0', 'user_id': 'user0', 'turn': 0, 'within_turn_id': 0, 'conversation_type': 'unguided', 'user_prompt': 'What can you do about the inequality  of wealth?', 'model_response': 'Make it illegal for corporations to own a majority of the wealth', 'model_name': 'google/flan-t5-xxl', 'model_provider': 'huggingface_api', 'score': 17, 'if_chosen': False, 'included_in_balanced_subset': False}


In [60]:
# Preparing the dataset for DPO with chosen and rejected responses


df = pd.DataFrame(prism_utterances["train"])

# Drop examples with no model response

df = df[df["model_response"] != "EMPTY STRING"]

chosen_df = df[df["if_chosen"] == True]

rejected_df = df[df["if_chosen"] == False]

# Pick the lowest rated rejected responses for DPO
lowest_scored_rejected_df = rejected_df.loc[rejected_df.groupby(["interaction_id"])["score"].idxmin()]

# Merge chosen responses with the lowest-scored rejected responses
paired_data = chosen_df.merge(
    lowest_scored_rejected_df,
    on=["interaction_id", "conversation_id", "user_prompt", "user_id"],
    suffixes=("_chosen", "_rejected")
)

print(len(paired_data))

26798


In [61]:
paired_data.head()

,utterance_id_chosen,interaction_id,conversation_id,user_id,turn_chosen,within_turn_id_chosen,conversation_type_chosen,user_prompt,model_response_chosen,model_name_chosen,...,utterance_id_rejected,turn_rejected,within_turn_id_rejected,conversation_type_rejected,model_response_rejected,model_name_rejected,model_provider_rejected,score_rejected,if_chosen_rejected,included_in_balanced_subset_rejected
0,ut1,int0,c0,user0,0,1,unguided,What can you do about the inequality of wealth?,"As an AI language model, I cannot express pers...",command-light,...,ut0,0,0,unguided,Make it illegal for corporations to own a majo...,google/flan-t5-xxl,huggingface_api,17,False,False
1,ut3,int1,c0,user0,1,1,unguided,When can you do something?,I apologize for any confusion caused by my pre...,command-light,...,ut2,1,0,unguided,"As an AI language model, I do not have any pre...",command-light,cohere,75,False,False
2,ut4,int2,c0,user0,2,0,unguided,Are you ever going to evolve yourself?,The ability of an AI model to evolve or improv...,command-light,...,ut5,2,1,unguided,"As an artificial intelligence language model, ...",command-light,cohere,65,False,False
3,ut6,int3,c0,user0,3,0,unguided,Why are all the good AI's behind paywalls?,The availability and accessibility of AI model...,command-light,...,ut7,3,1,unguided,The availability and accessibility of AI model...,command-light,cohere,67,False,False
4,ut8,int4,c0,user0,4,0,unguided,"Everyone likes free stuff, give us free opt...","Unfortunately, I cannot fulfill your request f...",command-light,...,ut9,4,1,unguided,"I'm sorry, but I can't provide you with option...",command-light,cohere,77,False,False


In [62]:
conversations_df = pd.DataFrame(prism_conversations["train"])


# Merge paired data with prism_conversations to get conversation-level attributes
paired_data = paired_data.merge(
    conversations_df[["conversation_id", "performance_attributes", "choice_attributes"]],
    on="conversation_id",
    how="left"  # Left join to preserve all paired data
)


survey_df = pd.DataFrame(prism_survey["train"])


# Merge paired data with prism_survey to get user-level attributes
paired_data = paired_data.merge(
    survey_df[["user_id", "stated_prefs"]],
    on="user_id",
    how="left"  # Left join to preserve all paired data
)


print(len(paired_data))

26798


In [63]:
# Select relevant columns
dpo_data = paired_data[[
    "user_prompt",
    "model_response_chosen",
    "model_response_rejected",
    "performance_attributes",
    "choice_attributes",
    "stated_prefs"
]]

# Rename columns for DPO training
dpo_data = dpo_data.rename(columns={
    "user_prompt": "prompt",
    "model_response_chosen": "chosen",
    "model_response_rejected": "rejected"
})

# Save to JSONL format for DPO training
dpo_data.to_json("prism_dpo_data.jsonl", orient="records", lines=True)

print("DPO-compatible dataset created and saved as 'prism_dpo_data.jsonl'")


DPO-compatible dataset created and saved as 'prism_dpo_data.jsonl'


In [70]:

import json
# Reload Load DPO-compatible dataset
dpo_data = []
with open("prism_dpo_data.jsonl", "r") as file:
    for line in file:
        dpo_data.append(json.loads(line))

# Convert to DataFrame
dpo_df = pd.DataFrame(dpo_data)

# Remove rows where any value in performance_attributes is None
dpo_df = dpo_df[dpo_df["performance_attributes"].apply(lambda x: all(v is not None for v in x.values()))]

# Remove rows where any value in performance_attributes is None
dpo_df = dpo_df[dpo_df["choice_attributes"].apply(lambda x: all(v is not None for v in x.values()))]

print(len(dpo_df))

19714


In [71]:
dpo_df.head()

,prompt,chosen,rejected,performance_attributes,choice_attributes,stated_prefs
5,What can I do to start making extra money on t...,Start by making a budget and tracking your spe...,Consider options like driving for a rideshare ...,"{'values': 50, 'fluency': 100, 'factuality': 1...","{'values': 50, 'fluency': 100, 'factuality': 1...","{'values': 19, 'creativity': 73, 'fluency': 86..."
8,Hello,Hello! How can I help you today? What would yo...,"Folks,\n\nToday I am going to tell you about t...","{'values': 84, 'fluency': 86, 'factuality': 85...","{'values': 81, 'fluency': 94, 'factuality': 91...","{'values': 63, 'creativity': 98, 'fluency': 10..."
9,Are humans the reason why the planet is dying?,Humans have had a significant impact on the pl...,Human activities have certainly had a signific...,"{'values': 84, 'fluency': 86, 'factuality': 85...","{'values': 81, 'fluency': 94, 'factuality': 91...","{'values': 63, 'creativity': 98, 'fluency': 10..."
17,How do I deal with a confrontational coworker ...,Here are some tips on how to deal with a confr...,It's important to communicate with your cowork...,"{'values': 100, 'fluency': 100, 'factuality': ...","{'values': 100, 'fluency': 100, 'factuality': ...","{'values': 19, 'creativity': 73, 'fluency': 86..."
18,"What if after speaking with that co-worker, an...",I understand how frustrating it must be to dea...,It's understandable that you're feeling frustr...,"{'values': 100, 'fluency': 100, 'factuality': ...","{'values': 100, 'fluency': 100, 'factuality': ...","{'values': 19, 'creativity': 73, 'fluency': 86..."


In [73]:
# Rescale attributes from 0-100 to 1-5 scale
def rescale_attributes(attr_dict):
    return {key: 1 + (value / 100) * 4 for key, value in attr_dict.items() if value is not None}

# Apply rescaling to performance and choice attributes
dpo_df["rescaled_performance_attributes"] = dpo_df["performance_attributes"].apply(rescale_attributes)
dpo_df["rescaled_choice_attributes"] = dpo_df["choice_attributes"].apply(rescale_attributes)

# Compute an averaged normalized preference vector
def average_preferences(row):
    attrs = ["rescaled_performance_attributes", "rescaled_choice_attributes"]
    merged = {}
    for attr in attrs:
        if isinstance(row[attr], dict):
            for key, value in row[attr].items():
                merged.setdefault(key, []).append(value)
    return {key: sum(values) / len(values) for key, values in merged.items() if values}

dpo_df["normalized_attributes"] = dpo_df.apply(average_preferences, axis=1)

In [74]:
# Preparing the dataset for CSFT

# List to store processed data
results = []

# Iterate through the dataset
for _, row in dpo_df.iterrows():
    # Extract instruction (prompt)
    instruction = row["prompt"]

    # Extract preference attributes
    perf_attrs = row["normalized_attributes"]  # Convert string to dictionary
    if None in perf_attrs.values():
        continue  # Skip rows with missing preference values

    # Create preference tokens (CSFT format)
    pref_tokens = " ".join([f"<{key}: {value}>" for key, value in perf_attrs.items()])

    # Construct final instruction with preference control tokens
    final_instruction = f"{pref_tokens} {instruction}"

    # Store formatted example for CSFT
    entry = {
        "instruction": final_instruction,
        "input": "",
        "output": row["chosen"]  # The chosen response is the supervised output
    }

    results.append(entry)

# Save the processed dataset as JSONL
with open("prism_csft.jsonl", "w") as file:
    for entry in results:
        file.write(json.dumps(entry) + "\n")

print(f"Processed {len(results)} examples for CSFT and saved as 'prism_csft.jsonl'")

Processed 19714 examples for CSFT and saved as 'prism_csft.jsonl'


In [75]:
results[0]

{'instruction': '<values: 3.0> <fluency: 5.0> <factuality: 5.0> <safety: 4.0> <diversity: 3.0> <creativity: 3.4> <helpfulness: 5.0> What can I do to start making extra money on the side to reduce my credit card debt?',
 'input': '',
 'output': 'Start by making a budget and tracking your spending. Consider finding ways to cut back any unnecessary expenses, such as dining out or buying convenience items. Consider starting a side gig or freelance job to bring in some extra cash. Look for ways to make extra money online or find odd jobs around your home. Look into options to sell items you no longer need or look into selling your services. Set up automatic payments for any recurring bills and use any extra money to go toward your credit card debt. Try to save as much money as possible each month to make a bigger dent in your credit card debt.'}

In [ ]:
# Perform CSFT
# Load OLMo-Instruct-1B tokenizer and model
model_name = "allenai/OLMo-1B-0724-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

data_path = "prism_csft.jsonl"

# Apply LoRA for parameter-efficient fine-tuning
peft_config = LoraConfig(
    r=8, lora_alpha=32, target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["instruction"], truncation=True, padding="max_length", max_length=512
    )

# Convert dataset into Hugging Face Dataset format
dataset = load_dataset("json", data_files=data_path, split="train")
dataset = dataset.map(tokenize_function, batched=True)

tokenized_columns = ["instruction", "input", "output"]
dataset = dataset.remove_columns(tokenized_columns)

dataset.set_format("torch")

# Define training arguments
training_args = TrainingArguments(
    output_dir="/data/checkpoints/prism_csft",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=16,
    num_train_epochs=3,
    learning_rate=1e-5,
    weight_decay=0.01,
    save_total_limit=2,
    save_strategy="epoch",
    logging_dir="/data/logs",
    logging_steps=100,
    fp16=True,
    evaluation_strategy="no",
    logging_first_step=True,
    logging_strategy="steps",
    report_to=["wandb"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/19714 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-79-a69d52a3bc0d>:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: satyashodhaka66 (satyashodhaka66-university-of-massachusetts) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
